In [1]:
from huggingface_hub import logout, notebook_login
# logout()
notebook_login()

README.md:   0%|          | 0.00/434 [00:00<?, ?B/s]

(…)-00000-of-00004-1949301f12e51c42.parquet:   0%|          | 0.00/109M [00:00<?, ?B/s]

(…)-00001-of-00004-a4cd9ef5eaa4f14a.parquet:   0%|          | 0.00/115M [00:00<?, ?B/s]

(…)-00002-of-00004-9d12bf5267a07ac2.parquet:   0%|          | 0.00/209M [00:00<?, ?B/s]

(…)-00003-of-00004-e49d8d0bebbbf654.parquet:   0%|          | 0.00/87.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1990915 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response'],
        num_rows: 1990915
    })
})

In [6]:
from datasets import load_dataset

sni = load_dataset("andersonbcdefg/supernatural-instructions-2m")
sni

diabetes_keyword = ['diabetes']

def extract_indices(dataset, keywords, columns=['prompt', 'response']):
    indices = []
    for i, row in enumerate(dataset):
        for col in columns:
            if any(keyword.lower() in str(row[col]).lower() for keyword in keywords):
                indices.append(i)
                break
    return indices

train_indices = extract_indices(sni['train'], diabetes_keyword)

filtered_train = sni["train"].filter(lambda row: any(keyword.lower() in str(row["prompt"]).lower() for keyword in diabetes_keyword))

print(filtered_train)

Filter:   0%|          | 0/1990915 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'response'],
    num_rows: 1383
})


In [9]:
type(filtered_train)

datasets.arrow_dataset.Dataset

In [7]:
filtered_train[0]

{'prompt': "In this task, you're given passages that contain mentions of names of people, places, or things. Some of these mentions refer to the same person, place, or thing. Your job is to write questions that evaluate one's understanding of such references. Good questions are expected to link pronouns (she, her, him, his, their, etc.) or other mentions to people, places, or things to which they may refer. Do not ask questions that can be answered correctly without understanding the paragraph or having multiple answers. Avoid questions that do not link phrases referring to the same entity. For each of your questions, the answer should be one or more phrases in the paragraph, and it should be unambiguous.\nPassage: Oklahoma was the 21st-largest recipient of medical funding from the federal government in 2005, with health-related federal expenditures in the state totaling $75,801,364; immunizations, bioterrorism preparedness, and health education were the top three most funded medical i

In [19]:
from datasets import Dataset
from tqdm import tqdm
import openai
import pandas as pd
import time
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

if not openai.api_key:
    raise ValueError("OpenAI API Key is missing! Please set OPENAI_API in .env file.")

client = openai.OpenAI()

def classify_task(prompt, response):
    system_prompt = """You are an AI assistant that classifies NLP tasks based on the given prompt and response. 
    Given a prompt and response, return only the most relevant NLP task category from the following list without any extra text:
    
    Named Entity Recognition (NER)
    Coreference Resolution
    Question Answering (QA)
    Summarization
    Text Classification
    Text Generation
    Relation Extraction (RE)
    Natural Language Inference (NLI)
    Information Extraction (IE)
    Commonsense Reasoning
    Other

    Ensure that identical or similar tasks receive exactly the same category name from the list.
    If unsure, return 'Other' without explanation.
    """

    user_input = f"Prompt: {prompt}\nResponse: {response}\nWhat is the most appropriate NLP task category?"

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input}
            ]
        )
        return response.choices[0].message.content.strip()

    except Exception as e:
        print(f"Error: {e}")
        return "Error"

df = pd.DataFrame(filtered_train)
df["task_category"] = "Pending"  

with open("task_classification_log_file.txt", "a") as log_file:
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing Task Classification"):
        try:
            result = classify_task(row["prompt"], row["response"])
            df.at[index, "task_category"] = result

            log_file.write(f"Index {index}: {result}\n")
            log_file.flush()  
            time.sleep(1)
        except Exception as e:
            print(f"Error at index {index}: {e}")
            df.at[index, "task_category"] = "Error"

filtered_train = Dataset.from_pandas(df)
print("Task classification complete. Dataset updated successfully.")


Processing Task Classification: 100%|██████████| 1383/1383 [35:58<00:00,  1.56s/it] 

Task classification complete. Dataset updated successfully.


In [1]:
filtered_train

NameError: name 'filtered_train' is not defined

In [23]:
from datasets import DatasetDict
from huggingface_hub import HfApi
import os

hf_token = os.getenv("HF_TOKEN_WRITE") 
if not hf_token:
    raise ValueError("Hugging Face API Token is missing! Please set HF_TOKEN in .env file.")

api = HfApi()

dataset_dict = DatasetDict({"test": filtered_train})

repo_name = "passionMan/diabetes_sni"  

dataset_dict.push_to_hub(repo_name, token=hf_token)

print(f"✅ Dataset successfully uploaded to: https://huggingface.co/datasets/{repo_name}")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

✅ Dataset successfully uploaded to: https://huggingface.co/datasets/passionMan/diabetes_sni


In [ ]:
import pandas as pd
from datasets import load_dataset

sni = load_dataset("passionMan/sni")
sni

df = pd.DataFrame(sni['test'])

README.md:   0%|          | 0.00/499 [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/575k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/800 [00:00<?, ? examples/s]

In [2]:
df.head(2)

,prompt,response,task_category
0,"In this task, you're given passages that conta...",In what city is the only Level 1 trauma center...,Question Answering (QA)
1,"In this task, you're given passages that conta...",What is the full name of the person who states...,Coreference Resolution


In [3]:
df['task_category'].value_counts()

task_category
Question Answering (QA)             517
Text Classification                 268
Text Generation                     161
Named Entity Recognition (NER)      125
Summarization                        99
Information Extraction (IE)          76
Relation Extraction (RE)             70
Commonsense Reasoning                22
Natural Language Inference (NLI)     19
Other                                11
Text Summarization                    4
Coreference Resolution                4
Question Generation                   3
Named Entity Recognition              2
Text Simplification                   1
Sentiment Analysis                    1
Name: count, dtype: int64

In [4]:
import pandas as pd

num_samples_main = 20  
num_samples_nli = 19   # NLI에서 샘플링할 개수

main_tasks = [
    "Question Answering (QA)", "Text Classification", "Text Generation",
    "Named Entity Recognition (NER)", "Summarization", "Information Extraction (IE)",
    "Relation Extraction (RE)", "Commonsense Reasoning"
]

sampled_dfs = []

for task in main_tasks:
    sampled_dfs.append(df[df["task_category"] == task].sample(n=num_samples_main, random_state=30, replace=False))

sampled_dfs.append(df[df["task_category"] == "Natural Language Inference (NLI)"].sample(n=num_samples_nli, random_state=42, replace=False))

df_sampled = pd.concat(sampled_dfs, ignore_index=True)

print(df_sampled.head())
print(f"✅ 총 샘플 개수: {len(df_sampled)}")


                                              prompt  \
0  You are given a background paragraph that desc...   
1  You are given a background paragraph that desc...   
2  You are given a background paragraph that desc...   
3  You are given a science question (easy-level) ...   
4  You are given a background paragraph that desc...   

                                            response            task_category  
0                                             Brown.  Question Answering (QA)  
1                                             Tammy.  Question Answering (QA)  
2  Which sibling did not spend a lot of time on t...  Question Answering (QA)  
3                                                  D  Question Answering (QA)  
4                                           group B.  Question Answering (QA)  
✅ 총 샘플 개수: 179


In [5]:
df_sampled

,prompt,response,task_category
0,You are given a background paragraph that desc...,Brown.,Question Answering (QA)
1,You are given a background paragraph that desc...,Tammy.,Question Answering (QA)
2,You are given a background paragraph that desc...,Which sibling did not spend a lot of time on t...,Question Answering (QA)
3,You are given a science question (easy-level) ...,D,Question Answering (QA)
4,You are given a background paragraph that desc...,group B.,Question Answering (QA)
...,...,...,...
174,"In this task, you are given an ambiguous quest...",Yes,Natural Language Inference (NLI)
175,"In this task, you are given an ambiguous quest...",Yes,Natural Language Inference (NLI)
176,"In this task, you are given an ambiguous quest...",Yes,Natural Language Inference (NLI)
177,The input is taken from a negotiation between ...,No,Natural Language Inference (NLI)


In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]= "1"

import openai
import pandas as pd
import os
import time
from tqdm import tqdm
from unsloth import FastLanguageModel

openai.api_key = os.getenv("OPENAI_API_KEY")
if not openai.api_key:
    raise ValueError("OpenAI API Key is missing! Please set OPENAI_API_KEY in .env file.")

client = openai.OpenAI()

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/data/jaesung/llm_for_diabetes/src/trial8/train/llama3_8B/outputs/real_seed_IFD_rIFD14/checkpoint-474",
    max_seq_length = 4096,
    dtype = None,
    load_in_4bit = False,
)

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token 

FastLanguageModel.for_inference(model) 

def generate_response(instruction_text, input_text, max_new_tokens=128):
    try:

        max_input_length = getattr(model.config, "max_position_embeddings", 4096)

        input_tokens = tokenizer(
            alpaca_prompt.format(instruction_text, input_text, ""),
            return_tensors="pt"
        ).to("cuda")

        input_length = input_tokens['input_ids'].shape[1]

        if input_length > max_input_length:
            print(f"[WARNING] Truncating input from {input_length} to {max_input_length} tokens.")
            input_text = tokenizer.decode(input_tokens['input_ids'][0, :max_input_length], skip_special_tokens=True)

        outputs = model.generate(
            **tokenizer(alpaca_prompt.format(instruction_text, input_text, ""), return_tensors="pt").to("cuda"),
            max_new_tokens=max_new_tokens,
            use_cache=True
        )

        decoded_outputs = tokenizer.batch_decode(outputs)
        response_texts = [output.split("### Response:\n")[-1].strip() for output in decoded_outputs]
        return response_texts[0].replace("<eot_id>", "")

    except Exception as e:
        print(f"[ERROR] Exception in response generation: {str(e)}")
        return "Error"

def get_gpt4o_response(prompt):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"[ERROR] Exception in GPT-4o response: {e}")
        return "Error"

df_sampled["llama3_output"] = "Pending"
df_sampled["gpt4o_output"] = "Pending"

for index, row in tqdm(df_sampled.iterrows(), total=len(df_sampled), desc="Generating Responses"):
    prompt = row["prompt"]

    df_sampled.at[index, "llama3_output"] = generate_response(instruction_text="", input_text=prompt)

    df_sampled.at[index, "gpt4o_output"] = get_gpt4o_response(prompt)

    time.sleep(1) 

print(df_sampled.head())

df_sampled.to_csv("df_sampled_with_outputs_bio2.csv", index=False)

print("✅ LLaMA3 & GPT-4o outputs successfully added and saved!")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-01 13:52:20 __init__.py:190] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A100-PCIE-40GB. Max memory: 39.394 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Unsloth 2025.2.12 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.
Generating Responses: 100%|██████████| 179/179 [12:55<00:00,  4.33s/it]

                                              prompt  \
0  You are given a background paragraph that desc...   
1  You are given a background paragraph that desc...   
2  You are given a background paragraph that desc...   
3  You are given a science question (easy-level) ...   
4  You are given a background paragraph that desc...   

                                            response            task_category  \
0                                             Brown.  Question Answering (QA)   
1                                             Tammy.  Question Answering (QA)   
2  Which sibling did not spend a lot of time on t...  Question Answering (QA)   
3                                                  D  Question Answering (QA)   
4                                           group B.  Question Answering (QA)   

                             llama3_output  \
0               Group Brown<|end_of_text|>   
1                       Pat<|end_of_text|>   
2  Why is Mary nearsighted?<|end_of_te

In [7]:
df_sampled.head(2)

,prompt,response,task_category,llama3_output,gpt4o_output
0,You are given a background paragraph that desc...,Brown.,Question Answering (QA),Group Brown<|end_of_text|>,Group Brown
1,You are given a background paragraph that desc...,Tammy.,Question Answering (QA),Pat<|end_of_text|>,Tammy


In [8]:
import openai
import pandas as pd
import os
import time
from tqdm import tqdm

openai.api_key = os.getenv("OPENAI_API_KEY")
if not openai.api_key:
    raise ValueError("OpenAI API Key is missing! Please set OPENAI_API_KEY in .env file.")

client = openai.OpenAI()

def evaluate_response(prompt, llama3_output, gpt4o_output):
    system_prompt = """You are an AI evaluator that compares two model-generated responses to a given prompt.
You will decide which response is better or if they are equally good.

Format your response as:
- "Llama3" if the Llama3 response is better
- "GPT-4o" if the GPT-4o response is better
- "Tie" if both are equally good

Here is the comparison:

Prompt: {prompt}
Llama3 Response: {llama3_output}
GPT-4o Response: {gpt4o_output}

Which response is better?"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": system_prompt.format(
                prompt=prompt,
                llama3_output=llama3_output,
                gpt4o_output=gpt4o_output
            )}]
        )
        return response.choices[0].message.content.strip()
    
    except Exception as e:
        print(f"[ERROR] Evaluation failed: {e}")
        return "Error"

df_sampled["evaluation"] = "Pending"

for index, row in tqdm(df_sampled.iterrows(), total=len(df_sampled), desc="Evaluating responses"):
    prompt = row["prompt"]
    llama3_output = row["llama3_output"]
    gpt4o_output = row["gpt4o_output"]

    df_sampled.at[index, "evaluation"] = evaluate_response(prompt, llama3_output, gpt4o_output)
    time.sleep(1)  # API Rate Limit 방지

task_win_counts = df_sampled.groupby("task_category")["evaluation"].value_counts().unstack().fillna(0)

task_win_counts["Total Matches"] = task_win_counts["Llama3"] + task_win_counts["GPT-4o"]
task_win_counts["Llama3 Win-Rate"] = task_win_counts["Llama3"] / task_win_counts["Total Matches"]

df_sampled.to_csv("df_sampled_with_outputs_evaluations_bio2.csv", index=False)


Evaluating responses: 100%|██████████| 179/179 [05:02<00:00,  1.69s/it]


In [9]:
task_win_counts

evaluation,"""GPT-4o""","""GPT-4o"" if the GPT-4o response is better","""GPT-4o"" is better","""GPT-4o"" is better.","""Llama3""","""Tie""","""Tie"" if both are equally good","- ""GPT-4o""","- ""GPT-4o"" if the GPT-4o response is better",GPT-4o,Llama3,Tie,Total Matches,Llama3 Win-Rate
task_category,,,,,,,,,,,,,,
Commonsense Reasoning,8.0,1.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,3.0,1.0,2.0,4.0,0.250000
Information Extraction (IE),3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,16.0,0.000000
Named Entity Recognition (NER),8.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,2.0,1.0,4.0,3.0,0.333333
Natural Language Inference (NLI),5.0,0.0,0.0,0.0,2.0,7.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,NaN
Question Answering (QA),6.0,0.0,1.0,2.0,1.0,3.0,1.0,0.0,0.0,2.0,1.0,3.0,3.0,0.333333
Relation Extraction (RE),4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,9.0,1.0,5.0,10.0,0.100000
Summarization,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,15.0,0.0,0.0,15.0,0.000000
Text Classification,7.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,4.0,0.0,5.0,4.0,0.000000
Text Generation,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,1.0,2.0,12.0,0.083333
